## Advanced RAG Pipeline
- RAGAS: https://www.youtube.com/watch?v=Anr1br0lLz8

In [2]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv('.env')
openai.api_base = os.getenv('OPENAI_ENDPOINT')
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_version = "2023-09-15-preview"
llm_model = 'gpt-35-turbo-jdrios'
emb_model = 'text-embedding-ada-002-jdrios'

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    "https://www.leal.co/usuarios"
)
documents = loader.load()

In [4]:
# Metadata
documents[0].metadata

{'source': 'https://www.leal.co/usuarios',
 'title': 'Usuarios Leal, puntos y Leal Coins',
 'description': 'Los puntos puedes redimirlos por premios exclusivos en la marca en la que acumulas, los Leal Coins son universales y puedes usarlos en miles de marcas físicas u online que tenemos para ti.',
 'language': 'No language found.'}

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

In [6]:
len(documents)

5

In [7]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=emb_model,
    azure_endpoint=os.getenv('OPENAI_ENDPOINT'),
)

In [8]:
# Create Vector Store
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(documents, embeddings)

In [9]:
# Creating Retriever
retriever = vector_store.as_retriever()

In [10]:
# How to retrieve content
retrieved_documents = retriever.invoke("Que son coins?")

In [11]:
for doc in retrieved_documents:
  print(doc)

page_content='Usuarios Leal, puntos y Leal Coins' metadata={'source': 'https://www.leal.co/usuarios', 'title': 'Usuarios Leal, puntos y Leal Coins', 'description': 'Los puntos puedes redimirlos por premios exclusivos en la marca en la que acumulas, los Leal Coins son universales y puedes usarlos en miles de marcas físicas u online que tenemos para ti.', 'language': 'No language found.'}
page_content='tu app Leal y entérate cuantos puntos te faltan para reclamar tu premio.¿Cuál es la diferencia entre puntos y Leal Coins?Los puntos puedes redimirlos por premios exclusivos en la marca en la que acumulas, los Leal Coins son universales y puedes usarlos en miles de marcas físicas u online que tenemos para ti.¿Dónde puedo usar mis Leal Coins?Usa tus Leal Coins para adquirir bonos de marcas como Mc Donalds, Crepes & Waffles o visita nuestros comercios Leal como Texaco, Buffalo Wings y úsalos para pagar. ¡Encuentra en nuestra app todas las opciones para usarlos!Actualiza tus datosMantén tus da

In [12]:
# Creatin Prompt
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'Redirigir...':
Context:
{context}
Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [13]:
from operator import itemgetter

from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = AzureChatOpenAI(model_name=llm_model, temperature=0, api_version="2023-09-15-preview", azure_endpoint=os.getenv('OPENAI_ENDPOINT'))

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [14]:
question = "Que son los Leal coins"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Los Leal Coins son universales y puedes usarlos en miles de marcas físicas u online que tienen para ti.


In [15]:
# Create Test Data
from langchain.document_loaders import DirectoryLoader
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.from_langchain(generator_llm=primary_qa_llm,critic_llm=primary_qa_llm,embeddings=embeddings)
testset = generator.generate_with_langchain_docs(documents, test_size=10, 
                                                 raise_exceptions=False, with_debugging_logs=False,
                                                 distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})   

c:\Users\Julián Ríos\Desktop\Repository\Machine Learning\Thesis Model\.venv_thesis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filename and doc_id are the same for all nodes.                 
Generating: 100%|██████████| 10/10 [16:45<00:00, 100.52s/it]


In [16]:
testset.test_data[1]

DataRow(question='What can you use Leal Coins for in the Leal app?', contexts=['tu app Leal y entérate cuantos puntos te faltan para reclamar tu premio.¿Cuál es la diferencia entre puntos y Leal Coins?Los puntos puedes redimirlos por premios exclusivos en la marca en la que acumulas, los Leal Coins son universales y puedes usarlos en miles de marcas físicas u online que tenemos para ti.¿Dónde puedo usar mis Leal Coins?Usa tus Leal Coins para adquirir bonos de marcas como Mc Donalds, Crepes & Waffles o visita nuestros comercios Leal como Texaco, Buffalo Wings y úsalos para pagar. ¡Encuentra en nuestra app todas las opciones para usarlos!Actualiza tus datosMantén tus datos actualizados para reclamar tus premios y enterarte de todas nuestras ofertas.Házlo aquí¡Empieza a'], ground_truth='Leal Coins can be used to acquire bonuses from various brands and pay at Leal partner stores like Texaco and Buffalo Wings. They are universal and can be used in thousands of physical and online brands. Th

In [17]:
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What can you do with the points you accumulate...,[tu app Leal y entérate cuantos puntos te falt...,You can redeem the points you accumulate in th...,simple,"[{'source': 'https://www.leal.co/usuarios', 't...",True
1,What can you use Leal Coins for in the Leal app?,[tu app Leal y entérate cuantos puntos te falt...,Leal Coins can be used to acquire bonuses from...,simple,"[{'source': 'https://www.leal.co/usuarios', 't...",True
2,What can you do with the points you accumulate...,[tu app Leal y entérate cuantos puntos te falt...,You can redeem the points you accumulate in th...,simple,"[{'source': 'https://www.leal.co/usuarios', 't...",True
3,What can you use Leal Coins for in the Leal app?,[tu app Leal y entérate cuantos puntos te falt...,Leal Coins can be used to acquire bonuses from...,simple,"[{'source': 'https://www.leal.co/usuarios', 't...",True
4,What can you use Leal Coins for in the Leal app?,[tu app Leal y entérate cuantos puntos te falt...,Leal Coins can be used to acquire bonuses from...,simple,"[{'source': 'https://www.leal.co/usuarios', 't...",True
5,What distinguishes points from Leal Coins in t...,[tu app Leal y entérate cuantos puntos te falt...,Points can be redeemed for exclusive rewards i...,reasoning,"[{'source': 'https://www.leal.co/usuarios', 't...",True
6,What distinguishes points from Leal Coins in t...,[tu app Leal y entérate cuantos puntos te falt...,Points can be redeemed for exclusive rewards i...,reasoning,"[{'source': 'https://www.leal.co/usuarios', 't...",True
7,What's the difference between Puntos and Leal ...,[tu app Leal y entérate cuantos puntos te falt...,Puntos can be redeemed for exclusive rewards f...,multi_context,"[{'source': 'https://www.leal.co/usuarios', 't...",True
8,What's the difference between Puntos and Leal ...,[tu app Leal y entérate cuantos puntos te falt...,Puntos can be redeemed for exclusive rewards f...,multi_context,"[{'source': 'https://www.leal.co/usuarios', 't...",True
9,What can you use Leal Coins for in the Leal app?,[tu app Leal y entérate cuantos puntos te falt...,Leal Coins can be used to acquire bonuses from...,simple,"[{'source': 'https://www.leal.co/usuarios', 't...",True


In [18]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [19]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [20]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [22]:
response_dataset[0]

{'question': 'What can you do with the points you accumulate in the Leal app?',
 'answer': 'You can redeem the points for exclusive prizes in the brand where you accumulate them.',
 'contexts': ['crédito Leal y gana cashback hasta de 5% con tus compras.¡Miles de opciones para que uses tus Leal Coins!Encuentra en nuestra app opciones desde $1.000 para disfrutar en las mejores marcas.Usa tus Leal Coins en nuestros comercios aliados mientras acumulas tus puntos.Preguntas FrecuentesResuelve tus dudas y empieza a ganar recompensas con tus compras usando Leal.¿Dónde puedo acumular puntos?Tenemos cientos de comercios donde puedes acumular puntos y redimirlos por premios increíbles. ¡Descarga Leal y descúbrelos!¿Cuáles son los premios que puedo reclamar?Tu marca favorita tiene una lista de premios que puedes redimir con tus puntos, conócela en tu app Leal y entérate cuantos puntos te faltan',
  'tu app Leal y entérate cuantos puntos te faltan para reclamar tu premio.¿Cuál es la diferencia entr

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [ ]:
results = evaluate(response_dataset, metrics)

In [21]:
results_df = results.to_pandas()
results_df

NameError: name 'results' is not defined

## Better implementation

In [ ]:
from langchain.retrievers import MultiQueryRetriever
advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

In [48]:
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
)
from ragas.metrics.critique import harmfulness

# list of metrics we're going to use
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    harmfulness,
]

In [44]:
import random
def chatbot_response(user_query, chatbot_name="Le A.I", company_name="Leal"):
  """
  This function simulates a friendly customer service chatbot conversation.

  Args:
      user_query: The user's question or input.
      chatbot_name: The name of the chatbot
      company_name: The name of your company

  Returns:
      A string representing the chatbot's response.
  """
  # Define conversation flow
  prompts = [
      {
          "START_SEQ": True,
          "USER_QUERY": None,
          "RESPONSE A": f"Hi there! I'm {chatbot_name}, your friendly customer service assistant for {company_name}. How can I help you today?",
          "RESPONSE B": f"Great to see you! Is there anything I can assist you with on this day?"
      },
      {
          "USER_QUERY": None,
          "RESPONSE A": f"I understand you're having trouble with {user_query}. Let's see what we can do to fix that.",
          "RESPONSE B": f"It sounds like you're looking for information about {user_query}. I'm happy to help you find what you need."
      },
      {
          "USER_QUERY": None,
          "RESPONSE A": "Here are a few things you can try: [List your solutions here]. Let me know if any of these work!",
          "RESPONSE B": f"I can definitely walk you through the steps for {user_query}. Would you like me to do that?",
          "RESPONSE C": f"No problem! I've found some helpful resources on {user_query} that you might find useful: [List your resources here]."
      },
      {
          "USER_QUERY": None,
          "RESPONSE A": "No worries at all! We'll get this figured out together.",
          "RESPONSE B": "That's a great question! Let me see if I can find an answer for you.",
          "RESPONSE C": f"I apologize for any inconvenience this may have caused. Is there anything else I can do to assist you today?"
      },
      {
          "USER_QUERY": None,
          "RESPONSE A": "If the issue seems complex, you can offer to connect the user to a human agent.",
          "RESPONSE B": f"It seems like your situation might require a bit more personalized attention. Would you like me to connect you with one of our customer service representatives?"
      },
      {
          "USER_QUERY": None,
          "RESPONSE A": "I hope this information was helpful! Is there anything else I can help you with today?",
          "RESPONSE B": "Glad I could be of assistance! Have a wonderful {day.name}!"
      },
      {
          "END_SEQ": True,
          "USER_QUERY": None,
          "RESPONSE A": None,
          "RESPONSE B": None
      }
  ]

  # Loop through conversation prompts
  current_prompt = 0
  while current_prompt < len(prompts):
    prompt = prompts[current_prompt]
    if prompt.get("USER_QUERY") is None or prompt.get("USER_QUERY") == user_query:
      response = random.choice([prompt.get(f"RESPONSE {char}") for char in "ABC" if prompt.get(f"RESPONSE {char}")==""])
      if response:
        print(response)
      if prompt.get("END_SEQ"):
        break
    current_prompt += 1

# Example usage
chatbot_response("I'm having trouble logging in.")

IndexError: Cannot choose from an empty sequence